V3 based on https://gemini.google.com/app/5a54aa16b879b0d3

## Creating the model
"This vignette is an example of modelling a decision tree using the rdecision package. It is based on the example given by Briggs (Box 2.3) which itself is based on a decision tree which compared oral Sumatriptan versus oral caffeine/Ergotamine for migraine. In this vignette, we consider the problem from the perspective of a provincial health department."

<figure>
<img src="img_Briggs2006_Box2-3.png" width="700" alt="Evans 1997 Figure 1"/>
<figcaption>BOX 2.3. Decision tree from Briggs et al., 2006.</figcaption>
</figure>

### Figure 1 from Evans, 1997
Below is Figure 1 from Evans, 1997 article showing the decision tree structure. 

<figure>
<img src="img_evans1997_fig01.drawio.png" width="100%" alt="Evans 1997 Figure 1"/>
<figcaption>FIGURE 1: From Evans, 1997, Figure 1.</figcaption>
</figure>

Figure 2 is the influence diagram of Figure 1. 

<figure>
<img src="img_evans1997_Influence_Diagram_V2.drawio.png" width="700" alt="Evans 1997 Figure 1"/>
<figcaption>FIGURE 2. Influence diagram derived from Evans, 1997, Figure 1.</figcaption>
</figure>

### Model variables
The following code defines the variables for cost, utility and effect that will be used in the model. There are 14 variables in total; 4 costs, 4 utilities and 6 probabilities.

For clarity, I coded some of the variables differently than in the original vignette. For example,

`p_norecurrence_relief_sumatriptan = 0.594` $= P(\text{no recurrence} \mid \text{relief}, \text{sumatriptan})$.

For example, 

`p_endures_norelief = 0.92` $= P(\text{endures} \mid \text{no relief})$ applies to both sumatriptan and caffeine/ergotamine treatment arms, so this variable appears in both arms.

## Set nodes, incoming edges, and states

In [3]:
using JuMP, HiGHS
using DecisionProgramming

### Set nodes with name, incoming edges, and states 

For example, consider,

`add_node!(diagram, ChanceNode("E", ["D", "R"], ["ED", "En"]))` 

- `"E"` is the name of the node
- `["D", "R"]` are the incoming edges from node D (Decision/Treatment) and node R (Relief vs No relief). This is called the "information set" of node M.
- `["ED", "En"]` are the states of node E: "ED" = Emeergency Department, "En" = Endure

I chose to use "M1" and "M0" because it is much easier to read in the code.

In [4]:
# 2. Initialize the Influence Diagram
diagram = InfluenceDiagram()

# 3. Define Nodes
# Decision: Treatment Options
add_node!(diagram, DecisionNode("D", [], ["D1", "D2"]))

# Chance: Relief (R depends on D)
add_node!(diagram, ChanceNode("R", ["D"], ["R1", "R2"]))

# Chance: Recurrence (N depends on D and R)
# We include "NA" for the case where Relief = No;  P(N = "NA" | R = "R2") = 1
add_node!(diagram, ChanceNode("N", ["D", "R"], ["N1", "N2", "NA"]))

# Chance: No Relief Outcome (E depends on R)
# E1 = endures attack, E2 = seeks help at ED; P(E = "NA" | R = "R1") = 1
# We include "NA" for the case where R = R1 (yes relief)
add_node!(diagram, ChanceNode("E", ["R"], ["E1", "E2", "NA"]))

# Chance: ED Outcome (H depends on E)
# H1 = relief, H2 = hospitalization
# We include "NA" for the case where E = E1 (endures attack); P(H = "NA" | E = "E1") = 1
add_node!(diagram, ChanceNode("H", ["E"], ["H1", "H2", "NA"]))

# 4. Define Probabilities (Conditional Probability Tables)
generate_arcs!(diagram)

OrderedCollections.OrderedDict{String, Utilities}()

### options for viewing diagram attributes  
`diagram.OPTION` where `OPTION` = `Nodes`, `Names`, `I_j`, `States`, `S`, `C`, `D`, or `V`.

In [5]:
diagram.I_j

OrderedCollections.OrderedDict{String, Vector{String}} with 5 entries:
  "D" => []
  "R" => ["D"]
  "N" => ["D", "R"]
  "E" => ["R"]
  "H" => ["E"]

## Assign probabilities to variables
For clarity, here is how the notation works:

REDO BELOW

- `p_E1` $=P(E = 1) = 0.20$
- `p_M1_E1_R1` $= P(M = 1 \mid E = 1, R = 1) = 0.05$
- `p_M1_E1_R0` $= P(M = 1 \mid E = 1, R = 0) = 0.33$
- `p_D1_M1` $= P(D = 1 \mid M = 1) = 0.0023$

In [6]:
# model variables for cost
c_sumatriptan = 16.10 #
c_caffeine = 1.32 #
c_ed = 63.16 #
c_hospital = 1093.0 #

# model variables for utility
u_N1_D1 = 1.0      # A  
u_N2_D1 = 0.9      # B
u_E1_D1 = - 0.3    # C 
u_H1_E2_D1 = 0.1   # D
u_H2_E2_D1 = - 0.3 # E
u_N1_D2 = 1.0      # F  
u_N2_D2 = 0.9      # G
u_E1_D2 = - 0.3    # H
u_H1_E2_D2 = 0.1   # I
u_H2_E2_D2 = - 0.3 # J

# model variables for effect
# Sumatriptan probabilities
p_R1_D1 = 0.558
p_R2_D1 = 1 - p_R1_D1
p_N1_D1 = 0.594 
p_N2_D1 = 1 - p_N1_D1 

# Caffeine/Ergotamine probabilities
p_R1_D2 = 0.379
p_R2_D2 = 1 - p_R1_D2
p_N1_D2 = 0.703 
p_N2_D2 = 1 - p_N1_D2

# both treatment arms  
p_E1_R2 = 0.92 
p_E2_R2 = 1 - p_E1_R2
p_H1_E2 = 0.998 
p_H2_E2 = 1 - p_H1_E2

p_norecurrence_relief_caffeine = 0.703 # e11; # e12 use NA_real_ to indicate complement

p_relief_ed = .998 # e15, e17; # e16, e18 use NA_real_ to indicate complement

0.998

### Create probability matrices and assign probabilities

In [7]:
#add_node!(diagram, DecisionNode("D", [], ["S", "C"])) # Decision: Sumatriptan (S) vs Coffeine/Ergot (C)
# not applicable 

In [8]:
#add_node!(diagram, ChanceNode("R", ["D"], ["R1", "R0"])) # R1 = relief, R0 = no relief  
X_R = ProbabilityMatrix(diagram, "R")

2×2 ProbabilityMatrix{2}:
 0.0  0.0
 0.0  0.0

In [9]:
X_R = ProbabilityMatrix(diagram, "R")
X_R["D1", "R1"] = p_R1_D1
X_R["D1", "R2"] = 1 - p_R1_D1
X_R["D2", "R1"] = p_R1_D2
X_R["D2", "R2"] = 1 - p_R1_D2
add_probabilities!(diagram, "R", X_R)

2×2 Probabilities{2}:
 0.558  0.442
 0.379  0.621

In [10]:
#add_node!(diagram, ChanceNode("N", ["D", "R"], ["N1", "N0", "NA"])) # N1 = non-recurrence, N0 = recurrence, NA = no action
X_N = ProbabilityMatrix(diagram, "N")

2×2×3 ProbabilityMatrix{3}:
[:, :, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 0.0  0.0

[:, :, 3] =
 0.0  0.0
 0.0  0.0

In [11]:
using DataFrames
d = ["D1", "D2"]
r = ["R1", "R2"]
n = ["N1", "N2", "NA"]
df = allcombinations(DataFrame, x=d, y=r, z=n)
sort!(df, [1 ,2, 3])
mat = Matrix(df)

12×3 Matrix{String}:
 "D1"  "R1"  "N1"
 "D1"  "R1"  "N2"
 "D1"  "R1"  "NA"
 "D1"  "R2"  "N1"
 "D1"  "R2"  "N2"
 "D1"  "R2"  "NA"
 "D2"  "R1"  "N1"
 "D2"  "R1"  "N2"
 "D2"  "R1"  "NA"
 "D2"  "R2"  "N1"
 "D2"  "R2"  "N2"
 "D2"  "R2"  "NA"

In [12]:
 X_N["D1",  "R1",  "N1"] = p_N1_D1
 X_N["D1",  "R1",  "N2"] = 1 - p_N1_D1
 X_N["D1",  "R1",  "NA"] = 0 # P(N = "NA" | R = "R2") = 0
 X_N["D1",  "R2",  "N1"] = 0 # P(N = "N1" | R = "R2") = 0
 X_N["D1",  "R2",  "N2"] = 0 # P(N = "N2" | R = "R2") = 0
 X_N["D1",  "R2",  "NA"] = 1 # P(N = "NA" | R = "R2") = 1
 X_N["D2",  "R1",  "N1"] = p_N1_D2
 X_N["D2",  "R1",  "N2"] = 1 - p_N1_D2
 X_N["D2",  "R1",  "NA"] = 0 # P(N = "NA" | R = "R2") = 0
 X_N["D2",  "R2",  "N1"] = 0 # P(N = "N1" | R = "R2") = 0
 X_N["D2",  "R2",  "N2"] = 0 # P(N = "N2" | R = "R2") = 0
 X_N["D2",  "R2",  "NA"] = 1 # P(N = "NA" | R = "R2") = 1
add_probabilities!(diagram, "N", X_N)

2×2×3 Probabilities{3}:
[:, :, 1] =
 0.594  0.0
 0.703  0.0

[:, :, 2] =
 0.406  0.0
 0.297  0.0

[:, :, 3] =
 0.0  1.0
 0.0  1.0

In [13]:
# Chance: No Relief Outcome (depends on R)
# E1 = endures attack, E2 = seeks help at ED; P(E = "NA" | R = R1) = 1
# We include "NA" for the case where R = R1 (yes relief)
X_E = ProbabilityMatrix(diagram, "E")

2×3 ProbabilityMatrix{2}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [15]:
X_E["R1", "E1"] = 0 # P(E = E1 | R = R1) = 0
X_E["R1", "E2"] = 0
X_E["R1", "NA"] = 1 # P(E = "NA" | R = R1) = 1
X_E["R2", "E1"] = p_E1_R2
X_E["R2", "E2"] = 1 - p_E1_R2
X_E["R2", "NA"] = 0 # P(E = "NA" | R = R2) = 0
add_probabilities!(diagram, "E", X_E)

2×3 Probabilities{2}:
 0.0   0.0   1.0
 0.92  0.08  0.0

In [16]:
# Chance: ED Outcome (H depends on E)
# H1 = relief, H2 = hospitalization
# We include "NA" for the case where E = E1 (endures attack); P(H = "NA" | E = "E1") = 1
X_H = ProbabilityMatrix(diagram, "H")

3×3 ProbabilityMatrix{2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [18]:
X_H["E1", "H1"] = 0 # P(H = "H1" | E = "E1") = 0
X_H["E1", "H2"] = 0 # P(H = "H2" | E = "E1") = 0
X_H["E1", "NA"] = 1 # P(H = "NA" | E = "E1") = 1
X_H["E2", "H1"] = p_H1_E2
X_H["E2", "H2"] = 1- p_H1_E2
X_H["E2", "NA"] = 0 # P(H = "NA" | E = "E2") = 0
X_H["NA", "H1"] = 0
X_H["NA", "H2"] = 0
X_H["NA", "NA"] = 1 # P(H = "NA" | E = "NA") = 1 # need to verify that this is correctly specified
add_probabilities!(diagram, "H", X_H)

3×3 Probabilities{2}:
 0.0    0.0    1.0
 0.998  0.002  0.0
 0.0    0.0    1.0

In [19]:
#add_node!(diagram, ValueNode("U1", ["D", "N"])) # 2x2
Y_U1 = UtilityMatrix(diagram, "U1")

DomainError: DomainError with Node U1 should be added as a node to the influence diagram.:


In [ ]:
Y_U1["D1", "N1"] = u_N1_D1 # A 
Y_U1["D1", "N2"] = u_N2_D1 # B
Y_U1["D2", "N1"] = u_N1_D2 # F
Y_U1["D2", "N2"] = u_N2_D2 # G
add_utilities!(diagram, "U1", Y_U1)

In [ ]:
#add_node!(diagram, ValueNode("U2", ["D", "E"])) # 2x2
Y_U2 = UtilityMatrix(diagram, "U2")

In [ ]:
Y_U2["D1", "E1"] = u_E1_D1 # C
Y_U2["D1", "E2"] = NaN
Y_U2["D2", "E1"] = u_E1_D2 # H
Y_U2["D2", "E2"] = NaN
add_utilities!(diagram, "U2", Y_U2)

In [ ]:
#add_node!(diagram, ValueNode("U3", ["D", "E", "H"])) # 2x2x2
Y_U3 = UtilityMatrix(diagram, "U3")

In [ ]:
Y_U3["D1", "E1", "H1"] = NaN
Y_U3["D1", "E1", "H2"] = NaN
Y_U3["D1", "E2", "H1"] = u_H1_E2_D1 # D
Y_U3["D1", "E2", "H2"] = u_H2_E2_D1 # E
Y_U3["D2", "E1", "H1"] = NaN
Y_U3["D2", "E1", "H2"] = NaN
Y_U3["D2", "E2", "H1"] = u_H1_E2_D2 # I
Y_U3["D2", "E2", "H2"] = u_H2_E2_D2 # J
add_utilities!(diagram, "U3", Y_U3)